In [ ]:
# --- CÉLULA 1: INSTALAÇÃO SEGURA ---
# Instala apenas o SDK do Gemini, sem quebrar o Colab
!pip install -q -U google-generativeai
print("✅ Instalação concluída. Pode rodar a próxima célula.")

# --- CÉLULA 2: CONEXÃO (LIMPA) ---
import google.generativeai as genai
from google.colab import userdata
import sys

# 1. Configura Chave
try:
    GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")
except Exception:
    GEMINI_API_KEY = None

if not GEMINI_API_KEY:
    GEMINI_API_KEY = input("Insira aqui sua chave")

genai.configure(api_key=GEMINI_API_KEY)

# 2. Seleciona o modelo que sabemos que funcionou (2.5-flash)
# O script anterior já nos confirmou que você tem acesso a ele.
MODEL_NAME = "models/gemini-2.5-flash"

print(f"⚙️ Configurando modelo: {MODEL_NAME}")
model = genai.GenerativeModel(MODEL_NAME)

try:
    resp = model.generate_content("Responda apenas: Conectado")
    print(f"✅ Status: {resp.text.strip()}")
except Exception as e:
    print(f"❌ Erro: {e}")

# --- CÉLULA 3: UPLOAD DE ARQUIVO LOCAL ---
from google.colab import files
import pandas as pd
import io

print("📂 Clique no botão abaixo para fazer upload do 'olist_products_dataset.csv'")
uploaded = files.upload()

# Verifica se o upload funcionou
if uploaded:
    # Pega o nome do primeiro arquivo enviado
    filename = list(uploaded.keys())[0]
    print(f"\n✅ Arquivo recebido: {filename}")

    # Lê o CSV direto da memória
    try:
        df = pd.read_csv(io.BytesIO(uploaded[filename]))
        print(f"📊 Dataset carregado: {len(df)} linhas.")

        # --- PREPARAÇÃO DOS DADOS (ETL) ---
        print("⚙️ Preparando descrições para o Gemini...")

        def criar_descricao(row):
            partes = []
            if pd.notna(row.get("product_category_name")):
                partes.append(f"Categoria: {row['product_category_name']}")
            if pd.notna(row.get("product_weight_g")):
                partes.append(f"Peso: {row['product_weight_g']}g")

            # Se não tiver nada, retorna vazio
            if not partes:
                return ""
            return " | ".join(partes)

        df["descricao_sintetica"] = df.apply(criar_descricao, axis=1)

        # Filtra produtos sem descrição ou muito curtos
        df_proc = df[df["descricao_sintetica"].str.len() > 5].copy()

        print("-" * 30)
        print(
            f"🚀 Tudo pronto! {len(df_proc)} produtos qualificados para enriquecimento."
        )

    except Exception as e:
        print(f"❌ Erro ao ler o CSV: {e}")
        print("Verifique se o arquivo é realmente um CSV separado por vírgulas.")
else:
    print("⚠️ Nenhum arquivo foi enviado. Rode a célula novamente.")

# Célula 4 — Engenharia de Prompt e Descrição Sintética


# 1. Preparação dos dados (Criando input para o LLM)
def criar_descricao_sintetica(row):
    partes = []
    # Tratamento básico de nulos e formatação
    if pd.notna(row.get("product_category_name")):
        partes.append(f"Categoria Original: {row['product_category_name']}")
    if pd.notna(row.get("product_weight_g")):
        partes.append(f"Peso: {row['product_weight_g']}g")
    if pd.notna(row.get("product_length_cm")):
        dims = f"{row['product_length_cm']}x{row['product_width_cm']}x{row['product_height_cm']}"
        partes.append(f"Dimensões (CxLxA): {dims} cm")

    return " | ".join(partes)


df["descricao_sintetica"] = df.apply(criar_descricao_sintetica, axis=1)
# Filtra apenas quem tem alguma informação
df_proc = df[df["descricao_sintetica"].str.len() > 10].copy()

# 2. Definição do Prompt do Sistema
# Dica de Data Engineer: Definir o schema esperado ajuda na validação posterior
SYSTEM_PROMPT = """
Você é um especialista em categorização de produtos e Master Data Management.
Analise os dados técnicos do produto e enriqueça as informações.

REGRAS DE SAÍDA:
1. Retorne APENAS um objeto JSON válido.
2. Não use blocos de código Markdown (```json).
3. Se não conseguir identificar, use null.

SCHEMA JSON ESPERADO:
{
  "categoria_normalizada": "string (ex: Eletrônicos, Casa e Decoração)",
  "subcategoria": "string",
  "tags": ["string", "string"],
  "perfil_logistico": "string (ex: Leve, Pesado, Sobredimensionado)",
  "publico_alvo": "string",
  "sugestao_nome_produto": "string (um nome comercial atrativo baseado nos dados)"
}
"""

print(f"Dados preparados. Linhas para processamento: {len(df_proc)}")

# --- CÉLULA 5: FUNÇÃO DE EXTRAÇÃO ROBUSTA (ANTI-429) ---
import time
import json
import google.generativeai as genai


def limpar_json(text):
    """Remove caracteres indesejados do Markdown."""
    text = text.strip()
    if text.startswith("```json"):
        text = text[7:]
    if text.startswith("```"):
        text = text[3:]
    if text.endswith("```"):
        text = text[:-3]
    return text


def extrair_features(descricao, tentativas_max=3):
    prompt = f"DADOS DO PRODUTO:\n{descricao}\n\nRetorne o JSON."

    # SYSTEM PROMPT (Recapitulando para garantir o contexto na função)
    sys_prompt = """
    Você é um especialista em e-commerce. Retorne APENAS um JSON válido.
    Campos: categoria_normalizada, subcategoria, tags (lista), perfil_logistico, publico_alvo, sugestao_nome_produto.
    Use null se não souber.
    """

    for i in range(tentativas_max):
        try:
            response = model.generate_content(
                [sys_prompt, prompt],
                generation_config={
                    "temperature": 0.1,
                    "response_mime_type": "application/json",
                },
            )
            return json.loads(limpar_json(response.text))

        except Exception as e:
            erro_str = str(e)

            # Se o erro for de COTA (429), precisamos esperar mais tempo
            if "429" in erro_str or "quota" in erro_str.lower():
                wait_time = 30  # Espera 30 segundos (Tier gratuito é restrito)
                print(
                    f"⏳ Cota atingida. Esperando {wait_time}s para tentar novamente..."
                )
                time.sleep(wait_time)
            else:
                # Outros erros (ex: JSON inválido), espera pouco
                print(f"⚠️ Erro na tentativa {i + 1}: {e}")
                time.sleep(2)

    return None  # Desiste após as tentativas


# --- CÉLULA 6: PROCESSAMENTO SEGURO (ANTI-429) ---
import time
import json
from tqdm import tqdm

# Limite pequeno para garantir sucesso no teste
SAMPLE_SIZE = 10
df_sample = df_proc.sample(n=min(SAMPLE_SIZE, len(df_proc)), random_state=42)
resultados = []

print(f"🚀 Processando {len(df_sample)} itens com Gemini 2.5...")


def extrair_features_safe(texto):
    prompt = f"Analise: {texto}. Retorne JSON com campos: categoria_normalizada, tags, publico_alvo."
    try:
        # Tenta chamar a API
        resp = model.generate_content(
            prompt, generation_config={"response_mime_type": "application/json"}
        )
        return json.loads(resp.text)
    except Exception as e:
        # Se der erro (provavelmente cota), retorna None mas não quebra o loop
        return None


for idx, row in tqdm(df_sample.iterrows(), total=len(df_sample)):
    dados = extrair_features_safe(row["descricao_sintetica"])

    if dados:
        resultados.append({**row.to_dict(), **dados})

    # PAUSA OBRIGATÓRIA PARA O GEMINI 2.5 (PLANO FREE)
    # Ele permite poucas chamadas por minuto. 10s é seguro.
    time.sleep(10)

print(f"\n✅ Sucesso! {len(resultados)} itens gerados.")
df_final = pd.DataFrame(resultados)
display(df_final.head())

# --- CÉLULA 7: EXPORTAÇÃO E DOWNLOAD ---
from google.colab import files
import json

# Define nomes dos arquivos
csv_filename = "olist_enriched_sample.csv"
json_filename = "olist_enriched_sample.json"

# Verifica qual dataframe está ativo (caso tenha rodado códigos diferentes)
if "df_final" in locals():
    df_export = df_final
elif "df_enriquecido" in locals():
    df_export = df_enriquecido
else:
    # Fallback para o último df usado
    df_export = pd.DataFrame(resultados)

# Salva em CSV
df_export.to_csv(csv_filename, index=False)
print(f"✅ CSV salvo: {csv_filename}")

# Salva em JSON (Formato orientado a registros, bom para NoSQL/MongoDB)
df_export.to_json(json_filename, orient="records", indent=2, force_ascii=False)
print(f"✅ JSON salvo: {json_filename}")

# Dispara o download no navegador
print("\n⬇️ Baixando arquivos para seu computador...")
files.download(csv_filename)
files.download(json_filename)